In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

# Define the ImageDataGenerator for loading and processing images with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_dir = r"E:\Research\ML - Machine Learning\Music Genre Classification\data\train"
validation_dir = r"E:\Research\ML - Machine Learning\Music Genre Classification\data\test"

# Generate the training data with data augmentation
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(288, 432),
    color_mode="rgba",
    class_mode='categorical',
    batch_size=128,
    subset="training"
)

# Generate the validation data
vali_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(288, 432),
    color_mode="rgba",
    class_mode='categorical',
    batch_size=128,
    subset="validation"
)

def build_model(input_shape):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(256, (2, 2), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(8, activation='softmax'))

    return model

# Model creation
input_shape = (288, 432, 4)
model = build_model(input_shape)

opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=30, validation_data=vali_generator)

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Get predictions for the validation dataset
y_val_true = vali_generator.classes
y_val_pred = model.predict(vali_generator)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

# Generate classification report and confusion matrix for validation
class_labels = list(vali_generator.class_indices.keys())
print("Validation Classification Report:")
print(classification_report(y_val_true, y_val_pred_classes, target_names=class_labels))

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val_true, y_val_pred_classes)
print(cm_val)


Found 4248 images belonging to 8 classes.
Found 320 images belonging to 8 classes.
Epoch 1/30
 1/34 [..............................] - ETA: 1:43:25 - loss: 2.8880 - accuracy: 0.1797

KeyboardInterrupt: 